In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
trades = pd.read_csv("senate_dataset.csv", parse_dates=['trade_date'])
prices = pd.read_csv("stocks_prices.csv", parse_dates=['date'])

In [4]:
trades['BS'] =  trades['type'].map({'Purchase': 1, 'Sale': -1})

returns = pd.DataFrame({'date': prices['date']})
for ticker in prices.columns.values[1:]:
    returns[ticker] = prices[ticker]/prices[ticker].shift(-1) - 1
returns.fillna(0, inplace = True)
returns.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

C:\Users\Menge\AppData\Local\Temp\ipykernel_25316\447790457.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  returns[ticker] = prices[ticker]/prices[ticker].shift(-1) - 1
C:\Users\Menge\AppData\Local\Temp\ipykernel_25316\447790457.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  returns[ticker] = prices[ticker]/prices[ticker].shift(-1) - 1
C:\Users\Menge\AppData\Local\Temp\ipykernel_25316\447790457.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

In [5]:
min_date = np.min(trades['trade_date'])
max_date = np.max(trades['trade_date'])

In [6]:
trades_pivot = pd.pivot_table(trades, values='BS',index=['trade_date'], columns=['ticker'], aggfunc=np.mean, fill_value=0)

In [8]:
senate_stocks = list(np.intersect1d(trades_pivot.columns.values, returns.columns.values))

market_returns = returns[returns['date'] <= max_date][min_date <= returns['date']]
market_returns.set_index('date', inplace=True)

senate_holdings = market_returns[senate_stocks]

for (ticker, s_ret) in senate_holdings.iteritems():
    curr_opinion = 0;
    for trade_date in senate_holdings.index:
        if (trade_date in trades_pivot.index):
            if (trades_pivot[ticker][trade_date] > 0):
                curr_opinion = 1
            elif (trades_pivot[ticker][trade_date] < 0):
                curr_opinion = 0
        senate_holdings.loc[trade_date, ticker] = curr_opinion

C:\Users\Menge\AppData\Local\Temp\ipykernel_25316\3106364943.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  market_returns = returns[returns['date'] <= max_date][min_date <= returns['date']]
C:\Users\Menge\AppData\Local\Temp\ipykernel_25316\3106364943.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senate_holdings.loc[trade_date, ticker] = curr_opinion


In [11]:
senate_holdings.to_csv('senate_picks.csv', index=False)
market_returns.to_csv('market_returns.csv', index=False)
returns.to_csv('returns.csv')

In [79]:
"""
senate_returns.reset_index(inplace=True)
market_returns.reset_index(inplace=True)

senate_returns['month'] = senate_returns['date'].dt.strftime('%Y/%m')
market_returns['month'] = market_returns['date'].dt.strftime('%Y/%m')
returns['month'] = returns['date'].dt.strftime('%Y/%m')

senate_monthly = senate_returns.groupby('month').sum()
market_monthly = market_returns.groupby('month').sum()

senate_monthly.to_csv('senate_monthly.csv')
market_monthly.to_csv('market_monthly.csv')

"""

C:\Users\Menge\AppData\Local\Temp\ipykernel_7572\4135070693.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senate_returns['month'] = senate_returns['date'].dt.strftime('%Y/%m')
